In [1]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
from datetime import date, timedelta
import time
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pycaret
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')
from pycaret.classification import *

In [3]:
train_err = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_err2.csv')
train_qui = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_qui2.csv')
train_prob = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_prob2.csv')

test_err = pd.read_csv('/content/drive/Shareddrives/dacon/404/test_err2.csv')
test_qui = pd.read_csv('/content/drive/Shareddrives/dacon/404/test_qui2.csv')

In [4]:
path = '/content/drive/Shareddrives/dacon/404/preprocessed/mk2_with_categorized/'

test_err_42_counts = pd.read_csv(path + '테스트_에러_42항목별_카운트.csv', index_col=['Unnamed: 0'])
train_err_42_counts = pd.read_csv(path + '트레인_에러_42항목별_카운트.csv', index_col=['Unnamed: 0'])

# 얘는 넣어봐야 의미가 없어서... 일단 제외
# test_err_code_frq = pd.read_csv(path + '테스트_에러_에러코드별_발생횟수.csv', index_col=['Unnamed: 0'])
# train_err_code_frq = pd.read_csv(path +'트레인_에러_에러코드별_발생횟수.csv', index_col=['Unnamed: 0'])

test_err_sum_byday = pd.read_csv(path +'테스트_에러발생_일일누적합계.csv', index_col=['Unnamed: 0'])
train_err_sum_byday = pd.read_csv(path +'트레인_에러발생_일일누적합계.csv', index_col=['Unnamed: 0'])

test_qui_frq = pd.read_csv(path +'테스트_퀼리티_항목별_많이등장.csv', index_col=['Unnamed: 0'])
train_qui_frq = pd.read_csv(path +'트레인_퀼리티_항목별_많이등장.csv', index_col=['Unnamed: 0'])

test_qui_sum_byday = pd.read_csv(path +'테스트_퀼리티로그_일일수집합계.csv', index_col=['Unnamed: 0'])
train_qui_sum_byday = pd.read_csv(path +'트레인_퀼리티로그_일일수집합계.csv', index_col=['Unnamed: 0'])

test_qui_sum_byquality = pd.read_csv(path +'테스트_퀼리티_각각카테고리_합산.csv', index_col=['index'])
train_qui_sum_byquality = pd.read_csv(path +'트레인_퀼리티_각각카테고리_합산.csv', index_col=['index'])

In [5]:
# 각 사용자가 42가지 에러타입을 몇 번씩 겪었는지 나타냄
train_err_42_counts

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
10000,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10001,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0
10002,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10003,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0
10004,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0.0,0.0,0.0,0.0,2.0,5.0,5.0,0.0,0.0,0.0,5.0,10.0,8.0,8.0,26.0,18.0,0.0,0.0,0.0,0.0,0.0,17.0,12.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,31.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,7.0,4.0
24996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24997,0.0,0.0,0.0,1.0,8.0,1.0,1.0,0.0,0.0,0.0,16.0,18.0,4.0,1.0,181.0,138.0,0.0,0.0,0.0,0.0,0.0,50.0,29.0,0.0,0.0,49.0,0.0,0.0,0.0,0.0,223.0,0.0,16.0,17.0,0.0,1.0,1.0,0.0,0.0,58.0,8.0,5.0
24998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,15.0,0.0,1.0,51.0,12.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,14.0,0.0,15.0,12.0,0.0,1.0,1.0,0.0,0.0,6.0,0.0,0.0


In [6]:
# 각 사용자가 퀼리티 1 부터 퀼리티 12까지 자주 겪은 퀼리티 로그를 나타냄
# _1은 가장 자주 겪은 로그이며, _2는 두 번째로 자주 겪은 로그임. (None은 2번째 로그가 없음, 한 가지 오류만 겪음)
train_qui_frq

################ 이거는 카테고리 기준 등장빈도이며... 이를 확률로 치환하는 새로운 방법을 시도해본다.

,q1_1,q1_2,q2_1,q2_2,q5_1,q5_2,q6_1,q6_2,q7_1,q7_2,q8_1,q8_2,q9_1,q9_2,q10_1,q10_2,q11_1,q11_2,q12_1,q12_2
10000,0.0,None,0.0,None,0.0,4.0,0.0,None,0.0,None,0.0,None,0.0,None,7.0,4.0,0.0,None,0.0,None
10002,0.0,-1.0,0.0,-1.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0,None,0.0,1.0,3.0,5.0,0.0,-1.0,0.0,None
10004,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,1.0,0.0,0.0,None,0.0,None,1.0,3.0,0.0,-1.0,0.0,None
10005,0.0,-1.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,1.0,0.0,0.0,None,0.0,None,4.0,5.0,0.0,-1.0,0.0,None
10006,0.0,None,0.0,None,0.0,4.0,0.0,1.0,0.0,1.0,0.0,None,0.0,None,4.0,None,0.0,None,0.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24990,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,None,0.0,None,0.0,None,1.0,None,0.0,-1.0,0.0,None
24992,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,None,0.0,None,0.0,None,5.0,3.0,0.0,-1.0,0.0,None
24993,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,None,0.0,None,0.0,None,5.0,3.0,0.0,-1.0,0.0,None
24995,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,None,0.0,None,0.0,None,3.0,2.0,0.0,-1.0,0.0,None


In [7]:
# 일단 자료들 날짜형식이나 콤마(,) 제거 등 간단한 전처리 진행

train_qui2 = train_qui.copy()
test_qui2 = test_qui.copy()
train_err2 = train_err.copy()
test_err2 = test_err.copy()

# 퀼리티 3,4는 값이 1밖에 없어서 의미가 없기 때문에 제외
# 펌웨어 버전은 이미 err 자료에 있기 때문에 중복으로 제외
# 퀼리티 0은 EDA 결과 퀼리티 2와 피어슨 상관계수 0.99를 보이기 때문에 0과 2 중에 하나만 골랐음
# 0의 결측치 Nan은 15%고 2의 결측치는 4%기 때문에 결측치가 적은 2를 남기고 0은 drop 시킨다.

## 퀼리티 6과 7도 0.81로 강한 상관관계를 보이고 있는데, 하나를 제외해야 하나 고민이 됨
train_qui2.drop(['fwver','quality_0', 'quality_3', 'quality_4'], axis =1, inplace=True)
test_qui2.drop(['fwver', 'quality_0', 'quality_3', 'quality_4'], axis =1, inplace=True)

test_qui2 = test_qui2.replace(',','', regex=True)
train_qui2 = train_qui2.replace(',','', regex=True)
test_qui2.loc[:,'quality_1':'quality_12'] = test_qui2.loc[:,'quality_1':'quality_12'].astype(float)
train_qui2.loc[:,'quality_1':'quality_12'] = train_qui2.loc[:,'quality_1':'quality_12'].astype(float)

train_qui2['time'] = pd.to_datetime(train_qui['time'], format = '%Y-%m-%d %H:%M:%S')
test_qui2['time'] = pd.to_datetime(test_qui['time'], format = '%Y-%m-%d %H:%M:%S')

train_err2['time'] = pd.to_datetime(train_err2['time'], format = '%Y-%m-%d %H:%M:%S')
test_err2['time'] = pd.to_datetime(test_err2['time'], format = '%Y-%m-%d %H:%M:%S')

train_err4 = train_err2.copy()
test_err4 = test_err2.copy()

In [8]:
# 퀼리티 임계치
# [-1, 0, 1] == -1 카테고리 -1, -1~0 =카테고리 0, 그 이상 = 카테고리 1
qualities = {}

qualities['quality_1'] = [-1,0, 1]
qualities['quality_2'] = [-1,0, 1]
qualities['quality_5'] = [-1,0,1,2,4, 10]
qualities['quality_6'] = [-1,0, 1]
qualities['quality_7'] = [0,1,100, 101]
qualities['quality_8'] = [0,1,2]
qualities['quality_9'] = [0,1,2]
qualities['quality_10'] = [0,1,2,3, 4, 5,7, 10 ,14,15] ##??
qualities['quality_11'] = [-1,0,1]
qualities['quality_12'] = [0,1,2]

## 모델 임계치
## 모델 0,1,2,3,4의 비율을 더하면 99.999% 이므로, 모델 5~8은 other model로 통합하는 것이 바람직 해 보임.

In [9]:
def quality_categorize(dataset):
  data = dataset.copy()
  for key in tqdm(qualities):
    intervals = qualities.get(key)
    for i in range(len(intervals)):
      if intervals[i] == intervals[-1]:
        data[key][data[key] >= intervals[i]] =  intervals[i]
      # elif intervals[i+1] - intervals[i] == 1:
      #   continue
      else:
        data[key][(data[key] >= intervals[i]) & (data[key] < intervals[i+1])] = intervals[i]
  return data

###### 그냥 쓰자니 불일치가 너무 많아서... 카테고리화 해서 확률로 전환한다.

train_qui3 = quality_categorize(train_qui2)
test_qui3 = quality_categorize(test_qui2)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [10]:
prob_list = list(set(train_prob.user_id))

train_qui3['problems'] = train_qui3.user_id.isin(prob_list)
train_qui3_tr = train_qui3[train_qui3['problems']].dropna()
train_qui3_fls = train_qui3[train_qui3['problems'] != True].dropna()

In [11]:
def probability(trainset, testset):
  train = trainset.copy()
  test = testset.copy()

  cols = list(train.loc[:,'quality_1':'quality_12'].columns)

  train['problems'] = train.user_id.isin(prob_list)
  train_tr = train[train['problems']]
  train_fls = train[train['problems'] != True]


  for col in tqdm(cols):
    temp_ori = train[col].value_counts()
    temp_tr = train_tr[col].value_counts()
    temp_fls = train_fls[col].value_counts()

    ori = pd.DataFrame(temp_ori)
    tr = pd.DataFrame(temp_tr)

    probs = tr/ori
    for ind in list(probs.index):
      train[col][train[col] == ind] = probs.loc[ind,:].values[0]
      test[col][test[col] == ind] = probs.loc[ind,:].values[0]
  train.drop(['problems'], axis = 1, inplace = True)
  return train, test, probs

In [12]:
train_qui_prob, test_qui_prob, prob = probability(train_qui3, test_qui3)

  0%|          | 0/10 [00:00<?, ?it/s]

In [13]:
# 일단 확률로 바꾼 값들을 같은 ID끼리 평균으로 묶는다.

train_qui_probs = train_qui_prob.groupby(by = 'user_id').mean()
test_qui_probs = test_qui_prob.groupby(by = 'user_id').mean()

In [14]:
#################### test_qui_frq 대신에 이걸 넣자. ########################
########## 넣어봤는데 그닥... 이번에는 단순 합계를 넣어보자  test_qui_sum_byquality


test_qui_probs

,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
user_id,,,,,,,,,,
30000,0.453202,0.470339,0.399646,0.456372,0.478145,0.463237,0.470470,0.390104,0.453996,0.468568
30001,0.471144,0.491062,0.449704,0.471127,0.465027,0.463237,0.470470,0.476673,0.471726,0.468568
30002,0.462173,0.480701,0.428165,0.462095,0.465027,0.463237,0.470470,0.490867,0.462861,0.468568
30003,0.453202,0.470339,0.399759,0.456372,0.472898,0.463237,0.470470,0.338923,0.453996,0.468568
30004,0.471144,0.491062,0.429772,0.474437,0.478145,0.463237,0.470470,0.366937,0.471726,0.468568
...,...,...,...,...,...,...,...,...,...,...
44990,0.461177,0.479549,0.429800,0.461092,0.465027,0.463237,0.470470,0.487934,0.461876,0.468568
44993,0.456193,0.475545,0.408818,0.459383,0.471586,0.463237,0.442244,0.402184,0.456951,0.468568
44994,0.518989,0.546323,0.520579,0.519299,0.465027,0.463237,0.470470,0.390104,0.519006,0.468568


In [15]:
### 펌웨어의 확률을 구하는 함수 ###
## 유진's 코드를 참조하여 좀더 빠르게 구동되도록 수정함.

def fw_probs(traindata, testdata):
  
  train_temp = traindata.drop_duplicates(subset = 'user_id')
  test_temp = testdata.drop_duplicates(subset = 'user_id')
  

  test_list = []
  train_temp['problems'] = train_temp['user_id'].isin(prob_list)
  train_tr = train_temp[train_temp['problems']]
  train_fls = train_temp[train_temp['problems'] != True]

  new_fw = ['03.11', '04.16', '04.22', '04.33', '04.73', '04.82', '05.15', '05.66', '8.5', '10']

  for fw in tqdm(new_fw):
    if fw == '10':
      ori = len(train_temp[(train_temp['fwver'].str.contains(fw)) & ~(train_temp['fwver'].str.contains('104')) & ~(train_temp['fwver'].str.contains('109'))])
      tr = len(train_tr[(train_tr['fwver'].str.contains(fw)) & ~(train_tr['fwver'].str.contains('104')) & ~(train_tr['fwver'].str.contains('109'))])
      fls = len(train_fls[(train_temp['fwver'].str.contains(fw)) & ~(train_fls['fwver'].str.contains('104')) & ~(train_fls['fwver'].str.contains('109'))])
      fw_st = [ori, tr, fls, tr/ori]
      test_list.append(fw_st)
    else:
      ori = len(train_temp[train_temp['fwver'].str.contains(fw)])
      tr = len(train_tr[train_tr['fwver'].str.contains(fw)])
      fls = len(train_fls[train_fls['fwver'].str.contains(fw)])
      fw_st = [ori, tr, fls, tr/ori]
      test_list.append(fw_st)

  fw_df = pd.DataFrame(test_list, index = new_fw, columns = ['원본', '불만고객', '고객', '불만확률'])

  train_temp['fw_prob'] = 0
  test_temp['fw_prob'] = 0

  for fw in tqdm(new_fw):
    if fw == '10':
      train_temp['fw_prob'][(train_temp['fwver'].str.contains(fw)) & ~(train_temp['fwver'].str.contains('104')) & ~(train_temp['fwver'].str.contains('109'))] = fw_df.loc[fw][3]
      test_temp['fw_prob'][(test_temp['fwver'].str.contains(fw)) & ~(test_temp['fwver'].str.contains('104')) & ~(test_temp['fwver'].str.contains('109'))] = fw_df.loc[fw][3]
    else:
      train_temp['fw_prob'][train_temp['fwver'].str.contains(fw)] = fw_df.loc[fw][3]
      test_temp['fw_prob'][test_temp['fwver'].str.contains(fw)] = fw_df.loc[fw][3]
  df = train_temp.loc[:,['user_id', 'fw_prob']]
  df_t = test_temp.loc[:,['user_id', 'fw_prob']]

  df.set_index('user_id', inplace = True)
  df_t.set_index('user_id', inplace = True)

  return df, df_t

In [16]:
train_fw, test_fw = fw_probs(train_err2, test_err2)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [17]:
# Error 데이터를 일일 단위로 쪼개서 각 사용자가 날짜별로 겪은 에러코드의 누계
# 0일은 10월 31일, 마지막은 11월 30일
train_err_sum_byday

,er_sum_0,er_sum_1,er_sum_2,er_sum_3,er_sum_4,er_sum_5,er_sum_6,er_sum_7,er_sum_8,er_sum_9,er_sum_10,er_sum_11,er_sum_12,er_sum_13,er_sum_14,er_sum_15,er_sum_16,er_sum_17,er_sum_18,er_sum_19,er_sum_20,er_sum_21,er_sum_22,er_sum_23,er_sum_24,er_sum_25,er_sum_26,er_sum_27,er_sum_28,er_sum_29,er_sum_30
10000,0,11,20,38,43,53,62,82,89,94,104,118,123,140,149,165,174,182,199,218,223,230,235,240,245,254,265,277,295,308,317
10001,0,11,61,90,138,180,209,222,246,299,351,374,413,421,462,486,501,531,556,583,628,653,671,689,716,727,785,2237,2275,2337,2365
10002,0,10,23,36,51,60,68,85,96,108,118,129,140,145,160,173,188,193,202,217,222,232,248,251,257,268,273,289,289,297,306
10003,0,9,23,33,38,54,59,61,72,77,96,120,126,132,134,136,151,157,164,179,184,191,212,223,240,246,254,275,286,300,306
10004,0,25,46,95,123,134,162,180,209,229,283,302,323,340,360,375,390,408,449,457,559,587,603,621,644,671,690,714,732,755,777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,41,58,76,76,100,103,133,155,172,194
24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
24997,0,9,43,77,141,163,194,218,248,274,292,301,320,339,377,408,418,445,464,496,525,550,579,621,654,695,718,748,779,813,826
24998,0,7,7,17,17,24,24,32,32,39,76,83,83,89,91,96,102,102,111,112,118,120,125,131,131,137,137,144,144,151,155


In [18]:
# Quality 데이터를 일일 단위로 쪼개서 퀼리티 로그가 사용자 별로 몇 번 수집되었나
# 누적, 마지막 11월 30일 자료는 최종 몇 번의 로그 수집이 있었는가를 의미함.
train_qui_sum_byday

,q_sum_0,q_sum_1,q_sum_2,q_sum_3,q_sum_4,q_sum_5,q_sum_6,q_sum_7,q_sum_8,q_sum_9,q_sum_10,q_sum_11,q_sum_12,q_sum_13,q_sum_14,q_sum_15,q_sum_16,q_sum_17,q_sum_18,q_sum_19,q_sum_20,q_sum_21,q_sum_22,q_sum_23,q_sum_24,q_sum_25,q_sum_26,q_sum_27,q_sum_28,q_sum_29,q_sum_30
10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0
10002,0.0,0.0,0.0,0.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,5.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,8.0
10004,0.0,0.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
10005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
10006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
24992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
24993,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0
24995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0


In [19]:
# train_err_42_counts, train_err_code_frq, train_qui_frq, train_err_sum_byday, train_qui_sum_byday
# test_err3, test_err_42_counts, test_err_code_frq, test_qui_frq, test_err_sum_byday, test_qui_sum_byday
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [20]:
test_qui_frq = test_qui_frq.astype(str)
train_qui_frq = train_qui_frq.astype(str)

In [21]:
# 24시간을 단순하게 평균내면, 23시와 1시의 평균이 이상해지기 때문에(실제로는 2시간 차이인데, 산술적으로 22시간 차이로 잡힘)
# 태양광 때 처럼 일단 코사인 시간으로 바꿔서 평균, 분산, 최빈값을 구한다.

def cos_time(dataset):
  dataset['cos_time'] = np.cos(2*np.pi*dataset.hour/24)
  return dataset

def cos_stats(dataset):
  dataset['hour'] = dataset['time'].dt.hour
  data = cos_time(dataset)[['user_id', 'cos_time']]

  temp = data.groupby(by = 'user_id')

  a = temp.mean()
  a.columns = ['mean']
  b = temp.std()
  b.columns = ['std']
  df = pd.concat([a,b], axis = 1) 
  return df

In [22]:
cos_train = cos_stats(train_err4)
cos_test = cos_stats(test_err4)

In [23]:
# errtype 1~42까지 확률로 치환하기.
train_err2['problems'] = train_err2['user_id'].isin(prob_list)
one_hot = pd.get_dummies(train_err2['errtype'], prefix = 'errtype')

temp = train_err2['problems']
temp_n = train_err2[['problems', 'user_id']]

In [24]:
def errtype_to_prob(trainset, testset):
  data = trainset.copy()
  t_data = testset.copy()
  data['problems'] = data['user_id'].isin(prob_list)
  one_hot = pd.get_dummies(data['errtype'])

  temp = data['problems']
  temp2 = pd.concat([temp, one_hot], axis = 1)

  temp2_tr = temp2[temp2['problems']].fillna(0)
  temp2_fls = temp2[temp2['problems'] != True].fillna(0)

  trues = pd.DataFrame(temp2_tr.mean()).transpose()
  falses = pd.DataFrame(temp2_fls.mean()).transpose()
  tru = trues.drop('problems', axis = 1)
  fls = falses.drop('problems', axis = 1)
  rslt = tru/(tru+fls)
  er_list = list(set(data.errtype))
  er_list.sort()
  for er in tqdm(er_list):
    data.errtype[data['errtype'] == er] =rslt.loc[0,er]
    t_data.errtype[t_data['errtype'] == er] =rslt.loc[0,er]
  return data, t_data

In [25]:
train, test = errtype_to_prob(train_err2, test_err2)

  0%|          | 0/41 [00:00<?, ?it/s]

In [30]:
train

,user_id,model_nm,fwver,errtype,errcode,time,problems
0,10000,model_3,05.15.2138,0.439427,1,2020-11-01 02:56:16,False
1,10000,model_3,05.15.2138,0.456132,1,2020-11-01 03:03:09,False
2,10000,model_3,05.15.2138,0.452552,1,2020-11-01 03:03:09,False
3,10000,model_3,05.15.2138,0.451911,1,2020-11-01 05:05:14,False
4,10000,model_3,05.15.2138,0.314866,0,2020-11-01 05:05:15,False
...,...,...,...,...,...,...,...
16554658,24999,model_3,05.15.2138,0.439427,1,2020-11-30 16:30:51,False
16554659,24999,model_3,05.15.2138,0.451911,1,2020-11-30 17:26:25,False
16554660,24999,model_3,05.15.2138,0.314866,0,2020-11-30 17:26:25,False
16554661,24999,model_3,05.15.2138,0.314866,0,2020-11-30 17:26:31,False


In [35]:
train_errtype_prob = train.groupby(by = 'user_id').mean()[['errtype']]
test_errtype_prob = test.groupby(by = 'user_id').mean()[['errtype']]

In [36]:
train_qui_sum_byquality

,quality_1 : -1,quality_1 : 0,quality_1 : 1,quality_2 : -1,quality_2 : 0,quality_2 : 1,quality_5 : -1,quality_5 : 0,quality_5 : 1,quality_5 : 2,quality_5 : 4,quality_5 : 10,quality_6 : -1,quality_6 : 0,quality_6 : 1,quality_7 : 0,quality_7 : 1,quality_7 : 100,quality_7 : 101,quality_8 : 0,quality_8 : 1,quality_8 : 2,quality_9 : 0,quality_9 : 1,quality_9 : 2,quality_10 : 0,quality_10 : 1,quality_10 : 2,quality_10 : 3,quality_10 : 4,quality_10 : 5,quality_10 : 7,quality_10 : 10,quality_10 : 14,quality_10 : 15,quality_11 : -1,quality_11 : 0,quality_11 : 1,quality_12 : 0,quality_12 : 1,quality_12 : 2
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000,0,24,0,0,24,0,0,22,0,0,2,0,0,24,0,24,0,0,0,24,0,0,24,0,0,0,0,0,0,12,0,12,0,0,0,0,24,0,24,0,0
10002,2,94,0,2,93,1,2,82,4,5,3,0,2,91,3,72,24,0,0,96,0,0,84,12,0,12,0,12,24,12,24,12,0,0,0,2,94,0,96,0,0
10004,2,22,0,2,22,0,2,19,2,1,0,0,2,21,1,12,12,0,0,24,0,0,24,0,0,0,12,0,12,0,0,0,0,0,0,2,22,0,24,0,0
10005,10,14,0,10,14,0,10,8,4,2,0,0,10,12,2,12,12,0,0,24,0,0,24,0,0,0,0,0,0,12,12,0,0,0,0,10,14,0,24,0,0
10006,0,36,0,0,36,0,0,33,0,0,3,0,0,35,1,24,12,0,0,36,0,0,36,0,0,0,0,0,0,36,0,0,0,0,0,0,36,0,36,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24990,8,16,0,8,16,0,8,14,2,0,0,0,8,16,0,24,0,0,0,24,0,0,24,0,0,0,24,0,0,0,0,0,0,0,0,8,16,0,24,0,0
24992,12,24,0,12,24,0,12,20,0,2,2,0,12,24,0,36,0,0,0,36,0,0,36,0,0,0,0,0,12,0,12,0,12,0,0,12,24,0,36,0,0
24993,7,17,0,7,17,0,7,15,0,1,1,0,7,17,0,24,0,0,0,24,0,0,24,0,0,0,0,0,12,0,12,0,0,0,0,7,17,0,24,0,0


In [48]:

trainset = pd.concat([train_fw, train_err_42_counts, train_qui_sum_byday, train_err_sum_byday, train_qui_probs, cos_train, train_errtype_prob], axis = 1) #train_err_sum_byday, train_qui_sum_byquality
testset = pd.concat([test_fw, test_err_42_counts, test_qui_sum_byday, test_err_sum_byday,test_qui_probs, cos_test, test_errtype_prob], axis = 1) #test_err_sum_byday, test_qui_sum_byquality

trainset = trainset.fillna(0)
testset = testset.fillna(0)

# trainset = trainset.drop(['model_nm', 'fwver'], axis = 1)
# testset = testset.drop(['model_nm','fwver'], axis = 1)

In [49]:
trainset2 = pd.DataFrame(scaler.fit_transform(trainset), columns=trainset.columns, index = trainset.index)
testset2 = pd.DataFrame(scaler.fit_transform(testset), columns=testset.columns, index = testset.index)
# trainset2 = trainset.copy()
# testset2 = testset.copy()

testset2

,fw_prob,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,er_sum_4,er_sum_5,er_sum_6,er_sum_7,er_sum_8,er_sum_9,er_sum_10,er_sum_11,er_sum_12,er_sum_13,er_sum_14,er_sum_15,er_sum_16,er_sum_17,er_sum_18,er_sum_19,er_sum_20,er_sum_21,er_sum_22,er_sum_23,er_sum_24,er_sum_25,er_sum_26,er_sum_27,er_sum_28,er_sum_29,er_sum_30,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,mean,std,errtype
30000,0.884039,0.0,0.0,0.000000,0.000000,0.004184,0.000594,0.000599,0.0,0.0,0.0,0.000441,0.007114,0.000000,0.000097,0.052467,0.029226,0.003112,0.042857,0.0,0.015152,0.0,0.035930,0.017937,0.001934,0.068966,0.123077,0.000000,0.000000,0.0,0.0,0.000691,0.000371,0.042553,0.000000,0.001075,0.200000,0.200000,0.000000,0.002237,...,0.001900,0.001603,0.002203,0.009041,0.008145,0.006964,0.006065,0.005826,0.005724,0.005760,0.006061,0.006755,0.007249,0.006639,0.006864,0.006543,0.006781,0.007014,0.007607,0.007217,0.007252,0.007638,0.008115,0.007830,0.007482,0.007078,0.006936,0.856072,0.860919,0.561515,0.873259,0.913739,0.849624,1.0,0.638596,0.874741,0.932043,0.543657,0.528088,0.739493
30001,0.442029,0.0,0.0,0.001939,0.000247,0.000202,0.002377,0.002397,0.0,0.0,0.0,0.000331,0.007114,0.004292,0.000195,0.041504,0.043444,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.092308,0.000015,0.000015,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.001184,0.000838,0.000681,0.000733,0.000722,0.000614,0.000544,0.000551,0.000582,0.000592,0.000659,0.000714,0.000703,0.000696,0.000683,0.000661,0.000689,0.000726,0.000762,0.000748,0.000774,0.000819,0.000825,0.000794,0.000755,0.000722,0.000716,0.889962,0.898850,0.631849,0.901492,0.888672,0.849624,1.0,0.780308,0.908903,0.932043,0.495897,0.615177,0.544272
30002,0.715664,0.0,0.0,0.000000,0.000000,0.005331,0.001188,0.000599,0.0,0.0,0.0,0.000441,0.007114,0.000000,0.000032,0.135474,0.105055,0.000000,0.000000,0.0,0.000000,0.0,0.002193,0.000549,0.000774,0.000000,0.153846,0.000000,0.000000,0.0,0.0,0.001915,0.000000,0.042553,0.000000,0.000000,0.066667,0.066667,0.000000,0.015660,...,0.002682,0.002206,0.001892,0.001855,0.001653,0.001480,0.001380,0.001632,0.001740,0.001832,0.001872,0.002047,0.002082,0.002001,0.001992,0.001922,0.002012,0.002100,0.002180,0.002118,0.002163,0.002212,0.002414,0.002391,0.002389,0.002321,0.002373,0.873017,0.879884,0.601585,0.884209,0.888672,0.849624,1.0,0.803543,0.891822,0.932043,0.455827,0.567579,0.630861
30003,0.715664,0.0,0.0,0.000000,0.000000,0.007220,0.001188,0.001198,0.0,0.0,0.0,0.000110,0.002223,0.001073,0.000227,0.006265,0.009479,0.000000,0.014286,0.0,0.015152,0.0,0.000110,0.000027,0.000000,0.000000,0.019231,0.000030,0.000030,0.0,0.0,0.001115,0.000000,0.010638,0.001014,0.003226,0.066667,0.066667,0.000000,0.000000,...,0.000737,0.000544,0.000542,0.000425,0.000611,0.000582,0.000555,0.000604,0.000607,0.000628,0.000629,0.000629,0.000593,0.000560,0.000540,0.000554,0.000797,0.000795,0.000861,0.000814,0.000836,0.000940,0.000964,0.000930,0.000928,0.000897,0.000936,0.856072,0.860919,0.561674,0.873259,0.903712,0.849624,1.0,0.554812,0.874741,0.932043,0.539711,0.590397,0.663297
30004,0.884039,0.0,0.0,0.000000,0.000005,0.000405,0.000594,0.000599,0.0,0.0,0.0,0.000276,0.005336,0.001073,0.000584,0.062647,0.052923,0.002075,0.000000,0.0,0.000000,0.0,0.000366,0.000110,0.000000,0.000000,0.053846,0.000015,0.000015,0.0,0.0,0.002764,0.028232,0.029255,0.000380,0.001075,0.066667,0.066667,0.007194,0.000000,...,0.003039,0.002118,0.001892,0.001656,0.001638,0.001467,0.001402,0.001443,0.001524,0.001601,0.001628,0.001682,0.001648,0.001606,0.001545,0.001711,0.001742,0.001782,0.001865,0.001821,0.001928,0.001973,0.002160,0.002212,0.002230,0.002210,0.002222,0.889962,0.898850,0.603843,0.907824,0.913739,0.849624,1.0,0.600671,0.908903,0.932043,0.498004,0.617789,0.644600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [29]:
# trainset = pd.concat([trainset, train_err_code_frq], axis = 1)
# testset = pd.concat([testset, test_err_code_frq], axis = 1)

In [50]:
trainset2 = trainset2.reset_index()
probs = list(set(train_prob.user_id))
trainset2['TARGET'] = trainset2['index'].apply(lambda x: 1 if x in probs else 0)
trainset2.set_index('index', inplace=True)

In [51]:
trainset2

,fw_prob,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,er_sum_5,er_sum_6,er_sum_7,er_sum_8,er_sum_9,er_sum_10,er_sum_11,er_sum_12,er_sum_13,er_sum_14,er_sum_15,er_sum_16,er_sum_17,er_sum_18,er_sum_19,er_sum_20,er_sum_21,er_sum_22,er_sum_23,er_sum_24,er_sum_25,er_sum_26,er_sum_27,er_sum_28,er_sum_29,er_sum_30,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,mean,std,errtype,TARGET
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000,0.442029,0.0,0.0,0.001098,0.000533,0.000000,0.000557,0.000372,0.0,0.0,0.000143,0.002047,0.002182,0.000172,0.000728,0.038663,0.039869,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.135021,0.000010,0.000010,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.001782,0.002044,0.002167,0.002350,0.002351,0.002174,0.002280,0.002197,0.002496,0.002635,0.002473,0.002376,0.002341,0.002377,0.002399,0.002260,0.001874,0.001739,0.001683,0.001216,0.001183,0.001222,0.001270,0.001342,0.001401,0.001422,0.856072,0.857253,0.570912,0.866926,0.867151,0.849624,1.000000,0.688523,0.874741,0.932043,0.592069,0.598977,0.254982,0
10001,0.554965,0.0,0.0,0.000000,0.000000,0.007986,0.000557,0.000372,0.0,0.0,0.000000,0.001364,0.001500,0.000172,0.001092,0.098952,0.083660,0.000000,0.097561,0.043478,0.130435,0.045455,0.047416,0.028345,0.001745,0.012821,0.092827,0.000000,0.000000,0.0,0.00,0.006644,0.0,0.054645,0.001457,0.0,0.166667,0.166667,0.000000,0.0,...,0.006051,0.006889,0.005866,0.006496,0.007478,0.007336,0.007227,0.007376,0.007505,0.008170,0.007284,0.006841,0.006830,0.006641,0.006415,0.006364,0.005321,0.004965,0.004832,0.003554,0.003385,0.003621,0.010258,0.010348,0.010630,0.010640,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.434389,0.697030,0.521619,1
10002,0.442029,0.0,0.0,0.000275,0.000676,0.000151,0.001114,0.000372,0.0,0.0,0.000020,0.001774,0.001909,0.000172,0.000291,0.034076,0.033987,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000698,0.000000,0.105485,0.000021,0.000021,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.002017,0.002241,0.002246,0.002535,0.002701,0.002466,0.002493,0.002500,0.002585,0.002829,0.002593,0.002567,0.002482,0.002413,0.002388,0.002250,0.001890,0.001835,0.001760,0.001276,0.001248,0.001259,0.001325,0.001315,0.001351,0.001373,0.858896,0.860933,0.569832,0.872181,0.873266,0.849624,0.942836,0.664475,0.877588,0.932043,0.548873,0.643147,0.213334,0
10003,0.554965,0.0,0.0,0.000000,0.000000,0.000301,0.000557,0.000372,0.0,0.0,0.000000,0.001228,0.001227,0.000000,0.000000,0.034076,0.019608,0.002174,0.000000,0.000000,0.000000,0.000000,0.001756,0.000717,0.000000,0.000000,0.248945,0.000000,0.000000,0.0,0.00,0.001728,0.0,0.043716,0.000000,0.0,0.166667,0.166667,0.008547,0.0,...,0.001815,0.001945,0.001612,0.001901,0.001926,0.002007,0.002319,0.002250,0.002353,0.002370,0.002038,0.002062,0.002019,0.001959,0.001970,0.001865,0.001556,0.001569,0.001564,0.001191,0.001145,0.001172,0.001261,0.001301,0.001365,0.001373,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.662895,0.546545,0.374172,0
10004,0.715664,0.0,0.0,0.000000,0.000005,0.000000,0.001670,0.001489,0.0,0.0,0.000000,0.002183,0.002591,0.000515,0.000364,0.093709,0.059477,0.000000,0.000000,0.000000,0.000000,0.000000,0.008781,0.004491,0.000000,0.000000,0.139241,0.000000,0.000000,0.0,0.00,0.004678,0.0,0.087432,0.000000,0.0,0.166667,0.166667,0.000000,0.0,...,0.004505,0.005339,0.004756,0.005519,0.005727,0.005915,0.005835,0.005768,0.006061,0.006366,0.005620,0.005325,0.005248,0.005363,0.005029,0.005665,0.004783,0.004461,0.004356,0.003197,0.003124,0.003183,0.003274,0.003330,0.003434,0.003493,0.867368,0.869843,0.574812,0.881614,0.879381,0.849624,1.000000,0.542102,0.886128,0.932043,0.580666,0.593177,0.429661,1
...,...,...,...,

In [52]:
X = trainset2.iloc[:,:-1]
y = pd.DataFrame(trainset2.iloc[:,-1])

In [ ]:
# best_5 = compare_models(sort = 'Accuracy', n_select = 3)

In [ ]:
# blended = blend_models(estimator_list = best_5, fold = 5, method = 'soft')

In [ ]:
# pred_holdout = predict_model(blended)

In [ ]:
# final_model = finalize_model(blended)

In [46]:
clf = setup(data = trainset2, target = 'TARGET') 


,Description,Value
0,session_id,2864
1,Target,TARGET
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(15000, 118)"
5,Missing Values,False
6,Numeric Features,115
7,Categorical Features,2
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
clf = setup(data = trainset2, target = 'TARGET') 
cat = create_model('catboost')

,Description,Value
0,session_id,4009
1,Target,TARGET
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(15000, 119)"
5,Missing Values,False
6,Numeric Features,116
7,Categorical Features,2
8,Ordinal Features,False
9,High Cardinality Features,False


IntProgress(value=0, description='Processing: ', max=4)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:40:58
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,CatBoost Classifier


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC


In [71]:
trainset2

,fw_prob,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,er_sum_4,er_sum_5,er_sum_6,er_sum_7,er_sum_8,er_sum_9,er_sum_10,er_sum_11,er_sum_12,er_sum_13,er_sum_14,er_sum_15,er_sum_16,er_sum_17,er_sum_18,er_sum_19,er_sum_20,er_sum_21,er_sum_22,er_sum_23,er_sum_24,er_sum_25,er_sum_26,er_sum_27,er_sum_28,er_sum_29,er_sum_30,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,mean,std,TARGET
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000,0.442029,0.0,0.0,0.001098,0.000533,0.000000,0.000557,0.000372,0.0,0.0,0.000143,0.002047,0.002182,0.000172,0.000728,0.038663,0.039869,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.135021,0.000010,0.000010,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.001485,0.001782,0.002044,0.002167,0.002350,0.002351,0.002174,0.002280,0.002197,0.002496,0.002635,0.002473,0.002376,0.002341,0.002377,0.002399,0.002260,0.001874,0.001739,0.001683,0.001216,0.001183,0.001222,0.001270,0.001342,0.001401,0.001422,0.856072,0.857253,0.570912,0.866926,0.867151,0.849624,1.000000,0.688523,0.874741,0.932043,0.592069,0.598977,0
10001,0.554965,0.0,0.0,0.000000,0.000000,0.007986,0.000557,0.000372,0.0,0.0,0.000000,0.001364,0.001500,0.000172,0.001092,0.098952,0.083660,0.000000,0.097561,0.043478,0.130435,0.045455,0.047416,0.028345,0.001745,0.012821,0.092827,0.000000,0.000000,0.0,0.00,0.006644,0.0,0.054645,0.001457,0.0,0.166667,0.166667,0.000000,0.0,...,0.004765,0.006051,0.006889,0.005866,0.006496,0.007478,0.007336,0.007227,0.007376,0.007505,0.008170,0.007284,0.006841,0.006830,0.006641,0.006415,0.006364,0.005321,0.004965,0.004832,0.003554,0.003385,0.003621,0.010258,0.010348,0.010630,0.010640,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.434389,0.697030,1
10002,0.442029,0.0,0.0,0.000275,0.000676,0.000151,0.001114,0.000372,0.0,0.0,0.000020,0.001774,0.001909,0.000172,0.000291,0.034076,0.033987,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000698,0.000000,0.105485,0.000021,0.000021,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.001761,0.002017,0.002241,0.002246,0.002535,0.002701,0.002466,0.002493,0.002500,0.002585,0.002829,0.002593,0.002567,0.002482,0.002413,0.002388,0.002250,0.001890,0.001835,0.001760,0.001276,0.001248,0.001259,0.001325,0.001315,0.001351,0.001373,0.858896,0.860933,0.569832,0.872181,0.873266,0.849624,0.942836,0.664475,0.877588,0.932043,0.548873,0.643147,0
10003,0.554965,0.0,0.0,0.000000,0.000000,0.000301,0.000557,0.000372,0.0,0.0,0.000000,0.001228,0.001227,0.000000,0.000000,0.034076,0.019608,0.002174,0.000000,0.000000,0.000000,0.000000,0.001756,0.000717,0.000000,0.000000,0.248945,0.000000,0.000000,0.0,0.00,0.001728,0.0,0.043716,0.000000,0.0,0.166667,0.166667,0.008547,0.0,...,0.001312,0.001815,0.001945,0.001612,0.001901,0.001926,0.002007,0.002319,0.002250,0.002353,0.002370,0.002038,0.002062,0.002019,0.001959,0.001970,0.001865,0.001556,0.001569,0.001564,0.001191,0.001145,0.001172,0.001261,0.001301,0.001365,0.001373,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.662895,0.546545,0
10004,0.715664,0.0,0.0,0.000000,0.000005,0.000000,0.001670,0.001489,0.0,0.0,0.000000,0.002183,0.002591,0.000515,0.000364,0.093709,0.059477,0.000000,0.000000,0.000000,0.000000,0.000000,0.008781,0.004491,0.000000,0.000000,0.139241,0.000000,0.000000,0.0,0.00,0.004678,0.0,0.087432,0.000000,0.0,0.166667,0.166667,0.000000,0.0,...,0.004247,0.004505,0.005339,0.004756,0.005519,0.005727,0.005915,0.005835,0.005768,0.006061,0.006366,0.005620,0.005325,0.005248,0.005363,0.005029,0.005665,0.004783,0.004461,0.004356,0.003197,0.003124,0.003183,0.003274,0.003330,0.003434,0.003493,0.867368,0.869843,0.574812,0.881614,0.879381,0.849624,1.000000,0.542102,0.886128,0.932043,0.580666,0.593177,1
...,...,...,...

In [56]:
# tuned_cat = tune_model(cat, optimize='AUC')

# 앙상블로 예측하고 싶으면 model = final_model, 캣부로 해보고 싶으면 model = tuned_cat
model = cat

predictions = predict_model(model, data = testset2)

In [43]:
x = []
for i in range(len(predictions['Score'])):
  if predictions['Label'][i+30000] ==1:
    x.append(predictions['Score'][i+30000])
  else:
    x.append(1-predictions['Score'][i+30000])

In [57]:
df = pd.DataFrame(model.feature_importances_, index = model.feature_names_, columns = ['feature'])

In [58]:
df = df.reset_index()

In [70]:
df.to_csv("F_impo2.csv", index = False)
!cp F_impo2.csv "drive/My Drive/"

In [ ]:
# sample_submssion = pd.read_csv('/content/drive/Shareddrives/dacon/404/sample_submission.csv')
# sample_submssion['problem'] = x
# sample_submssion.to_csv("확률_퀼리티_시간.csv", index = False)
# !cp 확률_퀼리티_시간.csv "drive/My Drive/"